In [0]:
# =============================================================================
# CAMADA SILVER - CARDS - MAGIC: THE GATHERING
# =============================================================================
"""
Script Python para processamento da tabela TB_FATO_SILVER_CARDS
Transformação e limpeza de dados da Bronze para Silver

USO DE SILVER_UTILS.PY:
- Centralização de funções comuns
- Padronização de processamento
- Redução de código duplicado
"""

# =============================================================================
# BIBLIOTECAS UTILIZADAS
# =============================================================================
import logging
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *

# =============================================================================
# CARREGAMENTO DO MÓDULO UTILITÁRIO
# =============================================================================
# Importar funções do silver_utils usando %run (Databricks) 
%run ./silver_utils

# =============================================================================
# CONFIGURAÇÃO INICIAL
# =============================================================================
def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

def setup_logging():
    """Configura logging para o script"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

def transform_cards_silver(df):
    """
    Transformação específica para tabela Cards
    Inclui lógica específica para Cards além das transformações padrão
    """
    if not df:
        return None
    
    logger = logging.getLogger(__name__)
    logger.info("Iniciando transformações específicas para Cards...")
    
    # Filtro temporal (últimos 5 anos)
    df = apply_temporal_filter(df, months_back=60)
    
    # Padronização de nomes (Title Case)
    df = apply_standard_cleaning(
        df,
        name_columns=["NME_CARD", "NME_ARTIST", "NME_RARITY", "NME_SET"],
        desc_columns=["DESC_CARD", "DESC_MANA_COST"],
        numeric_columns=["MANA_COST", "NME_POWER", "NME_TOUGHNESS"]
    )
    
    # Conversão de COD_SET para maiúsculas
    df = df.withColumn("COD_SET", upper(col("COD_SET")))
    
    # Derivação de NME_CARD_TYPE e DESC_CARD_TYPE
    def split_card_type(card_type):
        if card_type is None:
            return (None, None)
        if "planeswalker" in card_type.lower():
            return ("Planeswalker", card_type)
        if "—" in card_type:
            parts = card_type.split("—", 1)
            return (parts[0].strip(), parts[1].strip())
        return (card_type.strip(), "NA")
    
    split_card_type_udf = udf(split_card_type, StructType([
        StructField("NME_CARD_TYPE", StringType()),
        StructField("DESC_CARD_TYPE", StringType())
    ]))
    
    df = df.withColumn("_CARD_TYPE_STRUCT", split_card_type_udf(col("NME_CARD_TYPE")))
    df = df.withColumn("NME_CARD_TYPE", col("_CARD_TYPE_STRUCT.NME_CARD_TYPE"))
    df = df.withColumn("DESC_CARD_TYPE", col("_CARD_TYPE_STRUCT.DESC_CARD_TYPE"))
    df = df.drop("_CARD_TYPE_STRUCT")
    
    # Limpeza de colunas tipo array/JSON para string simples
    df = clean_array_json_columns(df, [
        "NME_PRINTINGS", "COD_COLORS", "COD_COLOR_IDENTITY", "DESC_SUBTYPES"
    ])
    
    # Tratamento específico para COD_COLORS
    df = df.withColumn(
        "COD_COLORS",
        when(col("COD_COLORS").isNull() | (col("COD_COLORS") == "") | (col("COD_COLORS") == '[""]'), lit("Colorless"))
        .otherwise(col("COD_COLORS"))
    )
    
    # NME_COLOR_CATEGORY
    df = df.withColumn(
        "NME_COLOR_CATEGORY",
        when(col("COD_COLORS") == "Colorless", "Colorless")
        .when(size(split(col("COD_COLORS"), ",")) == 1, "Mono")
        .when(size(split(col("COD_COLORS"), ",")) == 2, "Dual Color")
        .when(size(split(col("COD_COLORS"), ",")) >= 3, "Multicolor")
        .otherwise("Mono")
    )
    
    # QTY_COLORS: número de cores diferentes no custo de mana
    df = df.withColumn(
        "QTY_COLORS",
        when((col("DESC_MANA_COST").isNull()) | (col("DESC_MANA_COST") == "0") | (col("DESC_MANA_COST") == "1"), lit(0))
        .when(regexp_replace(col("DESC_MANA_COST"), "^[0-9X]+$", "") == "", lit(0))
        .otherwise(length(regexp_replace(upper(col("DESC_MANA_COST")), "[^WUBRG]", "")))
    )
    
    # Tratamento de DESC_TYPES e DESC_SUBTYPES
    df = df.withColumn("DESC_TYPES", when(col("DESC_TYPES").isNull() | (col("DESC_TYPES") == ""), lit("NA")).otherwise(col("DESC_TYPES")))
    df = df.withColumn(
        "DESC_SUBTYPES",
        when(col("DESC_SUBTYPES").isNull() | (col("DESC_SUBTYPES") == "") | (col("DESC_SUBTYPES") == '[""]'), lit("NA"))
        .otherwise(col("DESC_SUBTYPES"))
    )
    
    # Substituições em DESC_CARD
    desc_card_clean = regexp_replace(col("DESC_CARD"), r"\{W\}", "[White]")
    desc_card_clean = regexp_replace(desc_card_clean, r"\{U\}", "[Blue]")
    desc_card_clean = regexp_replace(desc_card_clean, r"\{B\}", "[Black]")
    desc_card_clean = regexp_replace(desc_card_clean, r"\{R\}", "[Red]")
    desc_card_clean = regexp_replace(desc_card_clean, r"\{G\}", "[Green]")
    desc_card_clean = regexp_replace(desc_card_clean, r"\{C\}", "[Colorless]")
    desc_card_clean = regexp_replace(desc_card_clean, r"\{X\}", "[X]")
    desc_card_clean = regexp_replace(desc_card_clean, r"\{T\}", "[Tap]")
    desc_card_clean = regexp_replace(desc_card_clean, r"\{Q\}", "[Untap]")
    desc_card_clean = regexp_replace(desc_card_clean, r"\{S\}", "[Snow]")
    desc_card_clean = regexp_replace(desc_card_clean, r"\{E\}", "[Energy]")
    
    df = df.withColumn("DESC_CARD", desc_card_clean)
    
    # Conversão de datas
    df = df.withColumn("DT_INGESTION", to_timestamp(col("DT_INGESTION")))
    
    # Adicionar colunas de particionamento
    df = add_partition_columns(df, "RELEASE_YEAR", "RELEASE_MONTH")
    
    logger.info(f"Transformação Cards concluída: {df.count()} registros")
    return df

# =============================================================================
# CONFIGURAÇÃO
# =============================================================================

# Configuração manual 
config = create_manual_config("magic_the_gathering", get_secret("s3_bucket"))

# Setup Unity Catalog
setup_unity_catalog(config['catalog_name'], config['schema_silver'])



In [0]:
# =============================================================================
# PROCESSAMENTO USANDO SILVER_UTILS
# =============================================================================
# Criar processor
processor = SilverTableProcessor("TB_FATO_SILVER_CARDS", config)

# Extração da Bronze
df_bronze = processor.extract_from_bronze("TB_BRONZE_CARDS")

# Aplicar transformação específica
df_silver = processor.transform_data(df_bronze, transform_cards_silver)

# Salvar na Silver com particionamento
processor.save_silver_table(
    df_silver, 
    partition_cols=["RELEASE_YEAR", "RELEASE_MONTH"]
)

# =============================================================================
# VALIDAÇÃO E LOGS
# =============================================================================
if df_silver:
    print(f"Processamento concluído com sucesso!")
    print(f"Registros processados: {df_silver.count()}")
    print(f"Colunas finais: {df_silver.columns}")
else:
    print("Falha no processamento - DataFrame vazio") 